# TensorFlow Data loading

In [4]:
import tensorflow as tf
import os
import numpy as np

In [ ]:
data_dir = '/Users/georgebrockman/code/georgebrockman/Autoenhance.ai/RoomDetection/images/training_data/'

In [ ]:
IMG_WIDTH, IMG_HEIGHT = 224, 224
IMG_SIZE = IMG_WIDTH, IMG_HEIGHT
batch_size = 32

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(IMG_WIDTH, IMG_HEIGHT),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(IMG_WIDTH, IMG_HEIGHT),
  batch_size=batch_size)

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Import the base model

In [ ]:
# instantiate the MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                             include_top=False,
                                             weights='imagenet')

In [ ]:
image_batch, label_batch = next(iter(train_ds)) # the next iteration in the dataset, so the first image
feature_batch = base_model(image_batch)
print(feature_batch.shape)

In [ ]:
# freeze the convolutional base
base_model.trainable=False

In [ ]:
# convert the features to a single 1280-element vector per image
global_av_layer = tf.keras.layers.GlobalAveragePooling2D() # averages over a 5x5 spatial 
feature_batch_av = global_av_layer(feature_batch)
print(feature_batch_av.shape)

In [ ]:
# apply a dense layer to convert these features into a single prediction per image
# no activation needed as the prediction will be treated as a logit (mapping of probabilities to Real Numbers)

pred_layer = tf.keras.layers.Dense(1)
pred_batch = pred_layer(feature_batch_av)
pred_batch.shape

## Build the model

In [ ]:
# augmentate the data 

data_aug = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

# rescale the pixel values to match the expected values of the MobileNetV2 model

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
# chain together data augmentation, rescaling, base_model and feature extractor layers useing the Keras Functional API

inputs = tf.keras.Input(shape=(224,224,3)) # image size and channels
# data augmentation layer
x = data_aug(inputs)
# preprocess, feed x into and reassign variable
x = preprocess_input(x)
# basemodel, set training =False for the BN layer
x = base_model(x, training=False)
# feature extraction
x = global_av_layer(x)
# add a dropout layer 
x = tf.keras.layers.Dropout(0.2)(x)

outputs = pred_layer(x)
model = tf.keras.Model(inputs, outputs)

## Compile the model

In [ ]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              # Only two linear outputs so use BinaryCrossentropy and logits =True
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

## Train the model

In [ ]:
initial_epochs = 10
history = model.fit(train_ds,
                    epochs=initial_epochs,
                    validation_data=val_ds)

In [ ]:
# balance the classes to help with the accuracy

# Using custom built loading function to build the model (with no valiadtion set)

In [5]:
def dataset_classifcation(path, resize_h, resize_w, train=True, limit=None):
    
    # list all paths to data classes except DS_Store
    class_folders = [f for f in sorted(os.listdir(path)) if not f.startswith('.')]
    # load images
    images = []
    classes = []
    for i, c in enumerate(class_folders):
        #images_per_class = sorted(os.path.join(path, c))
        images_per_class = [f for f in sorted(os.listdir(os.path.join(path, c))) if 'jpg' in f]
        
        for image_per_class in images_per_class:
            images.append(os.path.join(path, c, image_per_class))
            # the index will be the class label
            classes.append(i)

    images_tf = tf.data.Dataset.from_tensor_slices(images)
    classes_tf = tf.data.Dataset.from_tensor_slices(classes)
    # put two arrays together so that each image has its classifying label 
    dataset = tf.data.Dataset.zip((images_tf, classes_tf))
    
    @tf.function
    def read_images(image_path, class_type, mirrored=False):
        image = tf.io.read_file(image_path)
        image = tf.image.decode_jpeg(image)

        h, w, c = image.shape
        if not (h == resize_h and w == resize_w):
            image = tf.image.resize(
            image, [resize_h, resize_w], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
            # set all images shape to RGB
            image.set_shape((224, 224, 3))
#             print(image.shape)
        
        # change DType of image to float32
        image = tf.cast(image, tf.float32)
        class_type = tf.cast(class_type, tf.float32)
        
        # normalise the image pixels
        image = (image / 255.0)

        return image, class_type

    dataset = dataset.map(
        read_images,
        num_parallel_calls=tf.data.experimental.AUTOTUNE,
        deterministic=False)

    return dataset

In [6]:
# import the data 

path = '/Users/georgebrockman/code/georgebrockman/Autoenhance.ai/RoomDetection/images/training_data/'
train_dataset = dataset_classifcation(path, 224, 224)

In [7]:
train_dataset

<ParallelMapDataset shapes: ((224, 224, 3), ()), types: (tf.float32, tf.float32)>

In [8]:
IMG_WIDTH, IMG_HEIGHT = 224, 224
IMG_SIZE = IMG_WIDTH, IMG_HEIGHT
batch_size = 32

In [9]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).batch(32).prefetch(buffer_size=AUTOTUNE)

In [10]:
# import the base model
# instantiate the MobileNet V2
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                             include_top=False,
                                             weights='imagenet')

In [11]:
image_batch, label_batch = next(iter(train_dataset))# the next iteration in the dataset, so the first image
feature_batch = base_model(image_batch)
print(feature_batch.shape)

(32, 7, 7, 1280)


In [12]:
# freeze the convolutional base
base_model.trainable=False

In [13]:
# convert the features to a single 1280-element vector per image
global_av_layer = tf.keras.layers.GlobalAveragePooling2D() # averages over a 5x5 spatial 
feature_batch_av = global_av_layer(feature_batch)
print(feature_batch_av.shape)

(32, 1280)


In [14]:
# apply a dense layer to convert these features into a single prediction per image
# no activation needed as the prediction will be treated as a logit (mapping of probabilities to Real Numbers)

pred_layer = tf.keras.layers.Dense(1)
pred_batch = pred_layer(feature_batch_av)
pred_batch.shape

TensorShape([32, 1])

## Build the model volume 2

In [15]:
# augmentate the data 

data_aug = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.25),
    tf.keras.layers.experimental.preprocessing.RandomZoom(.5, .2),
    
])

# rescale the pixel values to match the expected values of the MobileNetV2 model

preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

In [16]:
# chain together data augmentation, rescaling, base_model and feature extractor layers useing the Keras Functional API

inputs = tf.keras.Input(shape=(224,224,3)) # image size and channels
# data augmentation layer
x = data_aug(inputs)
# preprocess, feed x into and reassign variable
x = preprocess_input(x)
# basemodel, set training =False for the BN layer
x = base_model(x, training=False)
# feature extraction
x = global_av_layer(x)
# add a dropout layer 
x = tf.keras.layers.Dropout(0.2)(x)

outputs = pred_layer(x)
model = tf.keras.Model(inputs, outputs)

In [17]:
# compile model again

In [18]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              # Only two linear outputs so use BinaryCrossentropy and logits =True
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [19]:
initial_epochs = 10
history = model.fit(train_dataset,
                    epochs=initial_epochs)

Epoch 1/10
 68/865 [=>............................] - ETA: 9:11 - loss: 0.6673 - accuracy: 0.8120

InvalidArgumentError:  Cannot batch tensors with different shapes in component 0. First element had shape [224,224,3] and element 25 had shape [224,224,1].
	 [[node IteratorGetNext (defined at <ipython-input-19-433f51138561>:3) ]] [Op:__inference_train_function_11712]

Function call stack:
train_function
